In [4]:
import os
import random
import threading
from time import sleep
import uuid
from concurrent.futures import ThreadPoolExecutor
from tenacity import retry, stop_after_attempt, wait_fixed
import chromadb

from chromadb.utils.embedding_functions import OpenAIEmbeddingFunction

# chroma run --path

client = chromadb.HttpClient()
ef = OpenAIEmbeddingFunction(api_key=os.environ.get("OPENAI_API_KEY"))
col = client.get_or_create_collection('chroma-qna', embedding_function=ef)

q = ef(["Italian football"])
@retry(stop=stop_after_attempt(3),wait=wait_fixed(2))
def query_docs(b):
    for _ in range(1):
        col.query(query_embeddings=q, n_results=2)
        sleep(random.uniform(0.01, 1.0))

def batch_generator(data, batch_size):
    for i in range(0, len(data), batch_size):
        yield data[i:i + batch_size]

ids_documents = [(f"{uuid.uuid4()}", f"document {i}",[0.1]*1536) for i in range(10000)]

with ThreadPoolExecutor(max_workers=30) as executor:
    for batch in batch_generator(ids_documents, 10):
        executor.submit(query_docs,batch)

    


In [5]:
with ThreadPoolExecutor(max_workers=50) as executor:
    for batch in batch_generator(ids_documents, 10):
        executor.submit(query_docs,batch)